# Day 2: Supply I

We talked today about how electricity markets work.

We will learn today how to build a simple model of an electricity market using **JuMP**.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.


We first load relevant libraries.

Compared to day 1, we will be adding the libraries `JuMP` and the non-linear solver `Ipopt`. We will also be using the clustering library `Clustering` and the package `Random` to control the randomness in the machine learning algorithm.

**Note:** I often prefer to use commercial solvers (Gurobi or CPLEX), which are available under an academic license. I use solvers that are readily available here without a license for simplicity and to ensure that everyone can access the code.


In [ ]:
#using Pkg
#Pkg.add([ "StatsPlots","StatsBase","Clustering","Random","JuMP","Ipopt","Printf"])

using DataFrames
using CSV
using Plots
using StatsPlots
using Statistics, StatsBase
using Clustering
using Random
using JuMP
using Ipopt
using Printf
using FixedEffectModels
	


We load the data using the CSV syntax (`CSV.read`) into a data frame called `df`.  Here we need to do some cleaning of the variables, rescaling and dropping missing entries.

Having a clean dataset will be very helpful for the clustering algorithm, which requires complete rows of data.


In [ ]:
# We read the data and clean it up a bit
df = CSV.read("data_jaere.csv", DataFrame)
df

In [ ]:
df = sort(df,["year","month","day","hour"])
df = dropmissing(df)
df.nuclear = df.nuclear/1000.0
df.hydro = df.hydro/1000.0
df.imports = df.imports/1000.0
df.q_commercial = df.q_commercial/1000.0
df.q_industrial = df.q_industrial/1000.0
df.q_residential = df.q_residential/1000.0
df.hydronuc = df.nuclear + df.hydro 
df = select(df,Not(["nuclear","hydro"]))


## Clustering

When modeling electricity markets, oftentimes the size of the problem can make the solver slow.

Here we will be using a clustering algorithm to come up with a (much) smaller synthetic dataset that we will use for the purposes of our main analysis.

**Note:** We ignore the time variables when we cluster. In our case, we'll transform a dataset of 43408 hours to 100 representatives hours. That is enough if you assume no correlation between hours.


In [ ]:
# define number of clusters
n = 100
# Clustering algorithms work in rows, so we need to transpose our data
X = transpose(Array(select(df,Between(:price,:hydronuc))));

# We scale variables to improve kmeans performance. For that, we take the mean and std of each row (dim=2) and you repeat it by the number of columns (rows in df)
Xs = (X.- repeat(mean(X,dims=2),1,nrow(df)))./repeat(std(X,dims=2),1,nrow(df)); 

#we set seed because kmeans picks random samples to generate clusters
Random.seed!(2020)
R = kmeans(Xs, n);

# Get the cluster centers rescaling again. These centers will be the new observations
M = R.centers .* repeat(std(X,dims=2),1,n) .+ repeat(mean(X,dims=2),1,n);  

# Assignments of each cluster
A = assignments(R)


In [ ]:
dfclust = DataFrame(transpose(M),
	["price", "imports", "q_commercial", "q_industrial", "q_residential", 
			"wind_cap", "solar_cap", "hydronuc"]);
    # the weights is defined by the number of old observations assigned to each cluster          
	dfclust.weights = counts(R);
	first(dfclust, 5)
	

We can compare the distribution of outcomes between the original dataset and the new dataset.

Here is an example with prices. The two distributions are very similar.


In [ ]:
histogram(df.price, fillalpha=.2, nbins=20, label="Data")
	histogram!(dfclust.price,weights=dfclust.weights, 
	fillalpha=.2, nbins=20, 
		label="Clusters")

It is also relatively well matched for the case for solar, although it is harder there.


In [ ]:
histogram(df.solar_cap, fillalpha=.2, nbins=20, label="Original")
	histogram!(dfclust.solar_cap, weights=dfclust.weights, fillalpha=.2, nbins=20, label="Clusters")

### Saving the output

It is useful to save the clustered data so that we can use it directly.


In [ ]:
CSV.write("data_jaere_clustered.csv", dfclust)


## Building the model

Now that we have clustered our data, we will build our model with the data that we have. 

The model that we will build today is a simplification from the original paper.

In the original paper, the model needed to solve for:
1. Endogenous retail prices (in a demand model, iterated to find equilibrium)
2. Endogenous investment (in same supply model, with more equations)

Here we will be simply building a simple model of market clearing.


Before building the model, we define some model parameters related to:

* Number and costs of different technologies (loaded from a small dataset)

* Elasticity of demand and imports


In [ ]:
tech = CSV.read("data_technology_simple.csv", DataFrame)


Here we assume zero marginal costs for renewable energy. `capUB` defines the maximum capacity of each power plant. For the existing ones, it defines the maximum production. For renewables, they will always produce at maximum capacity (precisely because of zero marginal costs) but production will depend on the capacity factors defined before.

To calibrate demand, one can use different strategies. Here we compute the slope for the demand curve that is consistent with the assumed elasticity of demand. 

Notice that this is a local elasticity approximation, but it has the advantage of being a linear demand curve, which is very attractive for the purposes of linear programming.

The demand is: $ q = a - b \ p $

So the elasticity becomes: $elas =  b  \frac{p}{q} $, which we set equal to an assumed parameter.

Once we have $b$, we can back out $a$. 

An analogous procedure is done for imports, but in this case, $ qm = am + bm \ p $


In [ ]:
dfclust.weights = dfclust.weights / sum(dfclust.weights);
	
	# Here only one demand type to make it easier
	dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;
	
    # Calibrate demand based on elasticities (using 0.1 here as only one final demand)
	elas = [.1, .2, .5, .3];
	dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
	dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

	# Calibrate imports (using elas 0.3)
    dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
    dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

In [ ]:
describe(dfclust)

### Non-linear solver

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$ \max \ CS - Costs $

$ \text{s.t.} \ \text{operational constraints, market clearing}. $


In [ ]:
function clear_market_max(data::DataFrame, tech::DataFrame; 
    wind_gw = 5.0, solar_gw = 2.0)

# We declare a model
model = Model(
    optimizer_with_attributes(
        Ipopt.Optimizer)
    );

# Set useful indexes
I = nrow(tech);  # number of techs
T = nrow(data);  # number of periods
S = 1;  # we will only be using one sector to keep things simple

# Variables to solve for
@variable(model, price[1:T]);
@variable(model, demand[1:T]);
@variable(model, imports[1:T]);
@variable(model, quantity[1:T, 1:I] >= 0);

# Maximize welfare including imports costs
@NLobjective(model, Max, sum(data.weights[t] * (
            (data.a[t] - demand[t]) * demand[t] / data.b[t] 
        + demand[t]^2/(2*data.b[t])
    - sum(tech.c[i] * quantity[t,i] 
                + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
    - (imports[t] - data.am[t])^2/(2 * data.bm[t])) for t=1:T));

# Market clearing
@constraint(model, [t=1:T], 
    demand[t] == data.a[t] - data.b[t] * price[t]);
@constraint(model, [t=1:T], 
    imports[t] == data.am[t] + data.bm[t] * price[t]);
@constraint(model, [t=1:T], 
    demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

# Constraints on output
@constraint(model, [t=1:T], 
    quantity[t,1] <= data.hydronuc[t]);	
@constraint(model, [t=1:T,i=2:3], 
    quantity[t,i] <= tech[i,"capUB"]);
@constraint(model, [t=1:T], 
    quantity[t,5] <= wind_gw * data.wind_cap[t]);
@constraint(model, [t=1:T], 
    quantity[t,6] <= solar_gw * data.solar_cap[t]);

# Solve model
optimize!(model);

status = @sprintf("%s", JuMP.termination_status(model));

if (status=="LOCALLY_SOLVED")
    p = JuMP.value.(price);
    avg_price = sum(p[t] * data.weights[t] for t=1:T);
    q = JuMP.value.(quantity);
    imp = JuMP.value.(imports);
    d = JuMP.value.(demand);
    cost = sum(data.weights[t] * (sum(tech.c[i] * q[t,i] 
            + tech.c2[i] * q[t,i]^2 / 2 for i=1:I) 
            + (imp[t] - data.am[t])^2/(2 * data.bm[t])) for t=1:T);
    results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
        "avg_price" => avg_price,
        "price" => p,
        "quantity" => q,
        "imports" => imp,
        "demand" => d,
        "cost" => cost);
    return results
else
    results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
    return results
end

end

In [ ]:
results = clear_market_max(dfclust, tech)


In [ ]:
results["avg_price"]

We can study the effects of an increase in renewable capacity

In [ ]:
results = clear_market_max(dfclust, tech, wind_gw = 7.0, solar_gw = 3.0)

In [ ]:
results["quantity"]

We can see the relation between renewable production and prices

In [ ]:
wind = results["quantity"][:,5]
price = results["price"]

In [ ]:
scatter(wind,price)

## Follow-up exercise

1. (*) The function is prepared to take several amounts of solar and wind. What are the impacts on prices as you increase solar and wind? Save prices for different values of wind or solar investment and plot them. Does your answer depend a lot on the number of clusters?
